In [255]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [256]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [257]:
df=pd.read_csv('/content/drive/MyDrive/vowel0.csv')
df

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,0,0,0,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,positive
1,0,0,0,-3.327,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,negative
2,0,0,0,-2.120,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,negative
3,0,0,0,-2.287,1.809,-1.498,1.012,-1.053,1.060,-0.567,0.235,-0.091,-0.795,negative
4,0,0,0,-2.598,1.938,-0.846,1.062,-1.633,0.764,0.394,-0.150,0.277,-0.396,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,1,14,1,-2.924,2.731,-1.138,0.066,0.100,0.683,0.162,-0.399,-0.882,0.827,negative
984,1,14,1,-3.239,3.083,-1.427,-0.202,-0.282,1.421,0.576,0.068,-0.914,0.147,negative
985,1,14,1,-3.753,3.605,-0.899,-0.747,-0.401,1.765,0.620,0.754,-0.835,-0.301,negative
986,1,14,1,-3.980,2.459,0.068,0.023,0.237,1.029,-0.189,0.521,-0.773,-0.500,negative


In [258]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0       positive
1       negative
2       negative
3       negative
4       negative
         ...    
983     negative
984     negative
985     negative
986     negative
987     negative
Name: Class, Length: 988, dtype: object

In [259]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,0,0,0,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1
1,0,0,0,-3.327,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,0
2,0,0,0,-2.120,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,0
3,0,0,0,-2.287,1.809,-1.498,1.012,-1.053,1.060,-0.567,0.235,-0.091,-0.795,0
4,0,0,0,-2.598,1.938,-0.846,1.062,-1.633,0.764,0.394,-0.150,0.277,-0.396,0


In [260]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(90, 13)

In [261]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(898, 13)

In [262]:
df_minor = pd.DataFrame(minority, columns =['TT','SpeakerNumber','Sex','F0','F1','F2','F3','F4','F5','F6','F7','F8','F9'])
df_minor

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,0.0,0.0,0.0,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814
1,0.0,0.0,0.0,-3.653,0.373,-0.600,1.705,-0.222,1.765,-0.353,0.537,-0.797,-0.813
2,0.0,0.0,0.0,-3.665,0.337,-0.641,1.791,-0.194,1.686,-0.359,0.570,-0.676,-0.841
3,0.0,0.0,0.0,-3.624,0.305,-0.708,1.758,-0.194,1.675,-0.273,0.561,-0.577,-0.843
4,0.0,0.0,0.0,-3.593,0.290,-0.782,1.707,-0.175,1.662,-0.137,0.493,-0.492,-0.926
...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101
894,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101
895,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101
896,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101


In [263]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,0.0,0.0,0.0,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1
1,0.0,0.0,0.0,-3.653,0.373,-0.600,1.705,-0.222,1.765,-0.353,0.537,-0.797,-0.813,1
2,0.0,0.0,0.0,-3.665,0.337,-0.641,1.791,-0.194,1.686,-0.359,0.570,-0.676,-0.841,1
3,0.0,0.0,0.0,-3.624,0.305,-0.708,1.758,-0.194,1.675,-0.273,0.561,-0.577,-0.843,1
4,0.0,0.0,0.0,-3.593,0.290,-0.782,1.707,-0.175,1.662,-0.137,0.493,-0.492,-0.926,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101,1
894,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101,1
895,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101,1
896,1.0,14.0,1.0,-1.093,2.319,1.431,2.377,1.831,2.327,1.059,1.453,0.551,1.101,1


In [264]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['TT','SpeakerNumber','Sex','F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','Class'])
df_major

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,0.0,0.0,0.0,-3.327,0.496,-0.694,1.365,-0.265,1.933,-0.363,0.510,-0.621,-0.488,0.0
1,0.0,0.0,0.0,-2.120,0.894,-1.576,0.147,-0.707,1.559,-0.579,0.676,-0.809,-0.049,0.0
2,0.0,0.0,0.0,-2.287,1.809,-1.498,1.012,-1.053,1.060,-0.567,0.235,-0.091,-0.795,0.0
3,0.0,0.0,0.0,-2.598,1.938,-0.846,1.062,-1.633,0.764,0.394,-0.150,0.277,-0.396,0.0
4,0.0,0.0,0.0,-2.852,1.914,-0.755,0.825,-1.588,0.855,0.217,-0.246,0.238,-0.365,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,1.0,14.0,1.0,-2.924,2.731,-1.138,0.066,0.100,0.683,0.162,-0.399,-0.882,0.827,0.0
894,1.0,14.0,1.0,-3.239,3.083,-1.427,-0.202,-0.282,1.421,0.576,0.068,-0.914,0.147,0.0
895,1.0,14.0,1.0,-3.753,3.605,-0.899,-0.747,-0.401,1.765,0.620,0.754,-0.835,-0.301,0.0
896,1.0,14.0,1.0,-3.980,2.459,0.068,0.023,0.237,1.029,-0.189,0.521,-0.773,-0.500,0.0


In [265]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,TT,SpeakerNumber,Sex,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,0.0,0.0,0.0,-3.639,0.418,-0.670,1.779,-0.168,1.627,-0.388,0.529,-0.874,-0.814,1.0
1,0.0,0.0,0.0,-3.653,0.373,-0.600,1.705,-0.222,1.765,-0.353,0.537,-0.797,-0.813,1.0
2,0.0,0.0,0.0,-3.665,0.337,-0.641,1.791,-0.194,1.686,-0.359,0.570,-0.676,-0.841,1.0
3,0.0,0.0,0.0,-3.624,0.305,-0.708,1.758,-0.194,1.675,-0.273,0.561,-0.577,-0.843,1.0
4,0.0,0.0,0.0,-3.593,0.290,-0.782,1.707,-0.175,1.662,-0.137,0.493,-0.492,-0.926,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,1.0,14.0,1.0,-2.924,2.731,-1.138,0.066,0.100,0.683,0.162,-0.399,-0.882,0.827,0.0
1792,1.0,14.0,1.0,-3.239,3.083,-1.427,-0.202,-0.282,1.421,0.576,0.068,-0.914,0.147,0.0
1793,1.0,14.0,1.0,-3.753,3.605,-0.899,-0.747,-0.401,1.765,0.620,0.754,-0.835,-0.301,0.0
1794,1.0,14.0,1.0,-3.980,2.459,0.068,0.023,0.237,1.029,-0.189,0.521,-0.773,-0.500,0.0


In [266]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [267]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [268]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.6611094128854437, 0.9456332428431012, 0.9456332428431012, 0.919746087272228, 0.7827330452331595]
AUC_SVM: [0.7333333333333334, 0.9720670391061452, 0.9720670391061452, 0.9583333333333333, 0.8722222222222222]
f1_SVM: [0.6363636363636364, 0.972972972972973, 0.972972972972973, 0.9597855227882037, 0.8861386138613861]
precision_SVM: [1.0, 0.9473684210526315, 0.9473684210526315, 0.9226804123711341, 0.7955555555555556]
recall_SVM: [0.4666666666666667, 1.0, 1.0, 1.0, 1.0]
mean of Gmean_s: 0.8509710062154067
mean of AUC_s: 0.9016045934202359
mean of f1_s: 0.8856467437918345
mean of precision_s: 0.9225945620063906
mean of recall_s: 0.8933333333333333
standard deviation of Gmean_s: 0.12585282456295385
standard deviation of AUC_s: 0.10286715115217679
standard deviation of f1_s: 0.1439536459613735
standard deviation of precision_s: 0.07641663193534665
standard deviation of recall_s: 0.23851391759997756


-------KNN--------
Gmean_KNN: [0.6636119710747722, 0.9047376955

In [269]:
list=['vowel0',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1.0,0.0,1.0,0.0,0.998828,0.000655,0.997651,0.001313
4,yeast1vs7,0.953475,0.075796,0.923035,0.116982,0.938317,0.063328,0.89309,0.093762,0.963953,0.080602,0.94367,0.125957
5,shuttle2vs4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,glass016vs2,0.925714,0.102718,0.885581,0.143202,0.885714,0.108327,0.827411,0.127491,0.591429,0.180871,0.723895,0.066047
7,glass016vs5,0.885714,0.137396,0.845013,0.164663,0.88,0.149352,0.84569,0.173688,0.951429,0.065153,0.914833,0.107251
8,pageblocks13vs4,0.921157,0.041699,0.858736,0.064528,0.966279,0.052842,0.939967,0.090831,0.781563,0.023842,0.673539,0.031343
9,yeast05679vs4,0.90693,0.11329,0.863197,0.148454,0.889167,0.098387,0.829291,0.128517,0.920559,0.118654,0.887151,0.160135


In [270]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')